In [23]:
from google.colab import drive

# Monta o Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install hiddenlayer

In [4]:
!git clone https://github.com/niccolo246/Advanced_nnUNet

Cloning into 'Advanced_nnUNet'...
remote: Enumerating objects: 586, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (391/391), done.
remote: Total 586 (delta 238), reused 523 (delta 190), pack-reused 0 (from 0)
Receiving objects: 100% (586/586), 704.49 KiB | 2.90 MiB/s, done.
Resolving deltas: 100% (238/238), done.


In [5]:
%cd

/root


In [6]:
%cd /content

/content


In [7]:
!dir

Advanced_nnUNet  drive	sample_data


In [8]:
%cd Advanced_nnUNet

/content/Advanced_nnUNet


In [9]:
!dir

nnunet		 nnUNet_LICENSE  scratch.py  setup.py
nnunet.egg-info  README.md	 setup.cfg


In [10]:
!pip install scikit-learn

In [11]:
!pip install torch>=1.6.0 tqdm dicom2nifti scikit-image>=0.14 medpy scipy batchgenerators>=0.23 numpy SimpleITK pandas requests nibabel tifffile matplotlib

In [12]:
!pip install -e .

Obtaining file:///content/Advanced_nnUNet
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Running setup.py develop for extended_nnunet


In [1]:
import os
os.environ["nnUNet_raw_data_base"] = "/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw_data_base"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_preprocessed"
os.environ["RESULTS_FOLDER"] = "/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50/nnUNet_results"

In [ ]:
!export nnUNet_raw_data_base="/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw_data_base"
!export nnUNet_preprocessed="/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_preprocessed"
!export RESULTS_FOLDER="/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_results"


In [20]:
import json
from collections import OrderedDict
from pathlib import Path
from batchgenerators.utilities.file_and_folder_operations import maybe_mkdir_p

# ------------------- CONFIGURAÇÃO -------------------
dataset_dir = Path("/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw_data_base/nnUNet_raw_data/Task001_MyBraTS")
imagesTr = dataset_dir / "imagesTr"
labelsTr = dataset_dir / "labelsTr"
imagesTs = dataset_dir / "imagesTs"

nnunet_dataset_name = "MyBraTS"
nnunet_dataset_id = 1
dataset_name = f"Task{nnunet_dataset_id:03d}_{nnunet_dataset_name}"
maybe_mkdir_p(dataset_dir)

# ------------------- CASOS DE TREINO E TESTE -------------------
train_cases = sorted({f.stem[:-9] for f in imagesTr.glob("*_0000.nii.gz")})
test_cases = sorted({f.stem[:-9] for f in imagesTs.glob("*_0000.nii.gz")})

# ------------------- LABELS E MODALIDADES -------------------
modalities = {
    "0": "PCA1",
    "1": "PCA2",
    "2": "PCA3",
}

labels_dict = {
    "0": "background",
    "1": "NETC",
    "2": "SNFH",
    "3": "ET",
    "4": "RC"
}

# ------------------- MONTAR LISTA DE TREINO -------------------
training_list = [
    {
        "image": f"./imagesTr/{case}.nii.gz",
        "label": f"./labelsTr/{case}.nii.gz"
    }
    for case in train_cases
]

# ------------------- MONTAR LISTA DE TESTE -------------------
test_list = [f"./imagesTs/{case}.nii.gz" for case in test_cases]

# ------------------- CRIAR JSON -------------------
json_dict = OrderedDict()
json_dict["name"] = dataset_name
json_dict["description"] = "Dataset MyBraTS subset 50 para nnU-Net v1"
json_dict["tensorImageSize"] = "4D"
json_dict["reference"] = "https://www.synapse.org/Synapse:syn53708249/wiki/627500"
json_dict["licence"] = "see https://www.synapse.org/Synapse:syn53708249/wiki/627508"
json_dict["release"] = "0.0"
json_dict["modality"] = modalities
json_dict["labels"] = labels_dict
json_dict["numTraining"] = len(train_cases)
json_dict["numTest"] = len(test_cases)
json_dict["training"] = training_list
json_dict["test"] = test_list

# ------------------- SALVAR -------------------
output_json_path = dataset_dir / "dataset.json"
with open(output_json_path, "w") as f:
    json.dump(json_dict, f, indent=4)

print(f"✅ JSON criado em: {output_json_path}")
print(f"Treino: {len(train_cases)} casos | Teste: {len(test_cases)} casos")


✅ JSON criado em: /content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw_data_base/nnUNet_raw_data/Task001_MyBraTS/dataset.json
Treino: 40 casos | Teste: 10 casos


In [25]:
!nnUNet_plan_and_preprocess -t 1 -pl3d ExperimentPlanner3DSpatialSingleAttentionUNet_v21 --verify_dataset_integrity



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Verifying training set
checking case BraTS-GLI-00008-100
checking case BraTS-GLI-00533-100
checking case BraTS-GLI-02068-100
checking case BraTS-GLI-02076-100
checking case BraTS-GLI-02160-100
checking case BraTS-GLI-02170-100
checking case BraTS-GLI-02185-100
checking case BraTS-GLI-02208-100
checking case BraTS-GLI-02218-100
checking case BraTS-GLI-02251-100
checking case BraTS-GLI-02338-100
checking case BraTS-GLI-02339-100
checking case BraTS-GLI-02375-100
checking case BraTS-GLI-02410-100
checking case BraTS-GLI-02420-100
checking case BraTS-GLI-02421-100
checking case BraTS-GLI-02478-100
checking case BraTS-GLI-02487-100
che

In [26]:
!nnUNet_train 3d_fullres nnUNetTrainerV2_SpatialSingleAttentionUNet 1 2 -p nnUNetPlans_SpatialSingleAttentionUNet_v2.1



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.architectural_variants.nnUNetTrainerV2_GenericSpatialSingleAttentionUNet.nnUNetTrainerV2_SpatialSingleAttentionUNet'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: 'PCA1', 1: 'PCA2', 2: 'PCA3'}
use_mask_for_norm OrderedDict({0: False, 1: False, 2: False})
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict({0: 'nonCT', 1: 'nonCT', 2: 'nonCT'})
stages...



In [4]:
import os
os.path.exists('/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50/nnUNet_preprocessed/Task001_MyBraTS/nnUNetData_plans_v2.1_stage0/BraTS-GLI-02817-100.npz')


True

In [7]:
os.path.exists('/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_preprocessed/Task001_MyBraTS/nnUNetPlans_SpatialSingleAttentionUNet_v2.1_plans_3D.pkl')


False